In [2]:
import io
import numpy as np
from torch import nn
import torch.onnx
import torch.nn as nn
import onnx
import torchinfo
from torchinfo import summary

In [12]:
class Concate(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(Concate, self).__init__()
        self.softmax = nn.Softmax(-1)
        self.conv_1 = nn.Conv2d(input_shape, 64, kernel_size=3, padding=1,stride=1)
        self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=64*32*32,
                      out_features=output_shape)
        )
    
    def forward(self, x):
        x = self.conv_1(x)
        x = self.softmax(x)
        x = self.max_pool(x)
        x = self.classifier(x)
        return x
    
model_0 = Concate(3,5)
model_0


Concate(
  (softmax): Softmax(dim=-1)
  (conv_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=65536, out_features=5, bias=True)
  )
)

In [13]:
summary(model_0, input_size=[1,3,64,64])

Layer (type:depth-idx)                   Output Shape              Param #
Concate                                  [1, 5]                    --
├─Conv2d: 1-1                            [1, 64, 64, 64]           1,792
├─Softmax: 1-2                           [1, 64, 64, 64]           --
├─MaxPool2d: 1-3                         [1, 64, 32, 32]           --
├─Sequential: 1-4                        [1, 5]                    --
│    └─Flatten: 2-1                      [1, 65536]                --
│    └─Linear: 2-2                       [1, 5]                    327,685
Total params: 329,477
Trainable params: 329,477
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 7.67
Input size (MB): 0.05
Forward/backward pass size (MB): 2.10
Params size (MB): 1.32
Estimated Total Size (MB): 3.46

In [14]:
x = torch.randn(1,3,64,64, requires_grad=True)
batch_size = 1
torch_out = model_0(x)
torch.onnx.export(model_0,
                  x,
                  "concate.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

